## Use the Keras Library to build a Multi-Layer Perceptron Model on the Boston Housing dataset

- The Boston Housing dataset comes with the Keras library so use Keras to import it into your notebook. 
- Normalize the data (all features should have roughly the same scale)
- Import the type of model and layers that you will need from Keras.
- Instantiate a model object and use `model.add()` to add layers to your model
- Since this is a regression model you will have a single output node in the final layer.
- Use activation functions that are appropriate for this task
- Compile your model
- Fit your model and report its accuracy in terms of Mean Squared Error
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Run this same data through a linear regression model. Which achieves higher accuracy?
- Do a little bit of feature engineering and see how that affects your neural network model. (you will need to change your model to accept more inputs)
- After feature engineering, which model sees a greater accuracy boost due to the new features?

In [120]:
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Reshape, Conv2D, AveragePooling2D, Flatten
from keras.layers import MaxPooling2D

from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd

In [106]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [107]:
train_data.shape

(404, 13)

## Normalize the Data

In [88]:
scaler = StandardScaler()

In [89]:
train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

In [90]:
train_data[0]

array([-0.27224633, -0.48361547, -0.43576161, -0.25683275, -0.1652266 ,
       -0.1764426 ,  0.81306188,  0.1166983 , -0.62624905, -0.59517003,
        1.14850044,  0.44807713,  0.8252202 ])

In [91]:
train_targets = train_targets.reshape(-1,1)
test_targets = test_targets.reshape(-1,1)

In [95]:
train_targets

array([[15.2],
       [42.3],
       [50. ],
       [21.1],
       [17.7],
       [18.5],
       [11.3],
       [15.6],
       [15.6],
       [14.4],
       [12.1],
       [17.9],
       [23.1],
       [19.9],
       [15.7],
       [ 8.8],
       [50. ],
       [22.5],
       [24.1],
       [27.5],
       [10.9],
       [30.8],
       [32.9],
       [24. ],
       [18.5],
       [13.3],
       [22.9],
       [34.7],
       [16.6],
       [17.5],
       [22.3],
       [16.1],
       [14.9],
       [23.1],
       [34.9],
       [25. ],
       [13.9],
       [13.1],
       [20.4],
       [20. ],
       [15.2],
       [24.7],
       [22.2],
       [16.7],
       [12.7],
       [15.6],
       [18.4],
       [21. ],
       [30.1],
       [15.1],
       [18.7],
       [ 9.6],
       [31.5],
       [24.8],
       [19.1],
       [22. ],
       [14.5],
       [11. ],
       [32. ],
       [29.4],
       [20.3],
       [24.4],
       [14.6],
       [19.5],
       [14.1],
       [14.3],
       [15

## Modeling

In [185]:
def build_model():
    model = Sequential()
    
    model.add(keras.layers.Dense(64, input_dim=13, activation="sigmoid"))
    model.add(keras.layers.Dense(64, activation="sigmoid"))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    
    
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['MSE'])
    return model

In [109]:
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print(f'Processing fold # {i}')
    val_data = train_data[i * num_val_samples: (i+1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i+1) * num_val_samples]
    
    partial_train_data = np.concatenate(
                            [train_data[:i * num_val_samples],
                            train_data[(i+1) * num_val_samples:]],
                            axis=0)
    partial_train_targets = np.concatenate(
                            [train_targets[:i * num_val_samples],
                            train_targets[(i+1)*num_val_samples:]],
                            axis=0)
    model = build_model()
    model.fit(partial_train_data,
              partial_train_targets,
              epochs=num_epochs,
              batch_size=1,
              verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append([val_mse, val_mae])

Processing fold # 0
Processing fold # 1
Processing fold # 2
Processing fold # 3


In [187]:
scores = []

In [188]:
model = build_model()
model.fit(partial_train_data,
          partial_train_targets,
          epochs=num_epochs,
          batch_size=1,
          verbose=0)
val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
scores.append([val_mse, val_mae])

In [189]:
scores

[[-371.1892289265548, 643.0856]]

### Evaluate Model

In [111]:
model = build_model()
model.fit(train_data, train_targets, epochs=100, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

102/102 [==============================] - 0s 3ms/sample - loss: -351.9829 - mean_squared_error: 570.7010


## Use the Keras Library to build an image recognition network using the Fashion-MNIST dataset (also comes with keras)

- Load and preprocess the image data similar to how we preprocessed the MNIST data in class.
- Make sure to one-hot encode your category labels
- Make sure to have your final layer have as many nodes as the number of classes that you want to predict.
- Try different hyperparameters. What is the highest accuracy that you are able to achieve.
- Use the history object that is returned from model.fit to make graphs of the model's loss or train/validation accuracies by epoch. 
- Remember that neural networks fall prey to randomness so you may need to run your model multiple times (or use Cross Validation) in order to tell if a change to a hyperparameter is truly producing better results.

In [175]:
import matplotlib.pyplot as plt
import tensorflow

In [176]:
from keras.datasets import mnist

(train_data, train_targets), (test_data, test_targets) = mnist.load_data()

In [199]:
train_data.shape[0]

60000

In [178]:
train_data[0].shape

(28, 28)

In [203]:
trainX2 = train_data.reshape((train_data.shape[0], 28, 28, 1))
testX2 = test_data.reshape((test_data.shape[0], 28, 28, 1))

In [204]:
model = Sequential()
#model.add(keras.layers.Reshape(target_shape = (28,28),input_shape=(784,)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='sigmoid'))
model.add(keras.layers.Dense(64, activation='sigmoid'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [208]:
adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model.fit(trainX2, train_targets, epochs=30, batch_size=64)
y_pred = model.predict_classes(testX2)


Epoch 1/30
60000/60000 [==============================] - 4s 67us/sample - loss: 159.8663 - acc: 0.1131
Epoch 2/30
60000/60000 [==============================] - 4s 59us/sample - loss: 159.8657 - acc: 0.0053
Epoch 3/30
60000/60000 [==============================] - 3s 56us/sample - loss: 159.8657 - acc: 0.0640
Epoch 4/30
60000/60000 [==============================] - 3s 55us/sample - loss: 159.8657 - acc: 0.1845
Epoch 5/30
60000/60000 [==============================] - 3s 54us/sample - loss: 159.8658 - acc: 0.1280
Epoch 6/30
60000/60000 [==============================] - 3s 56us/sample - loss: 159.8657 - acc: 0.1195
Epoch 7/30
60000/60000 [==============================] - 3s 56us/sample - loss: 159.8657 - acc: 0.1792
Epoch 8/30
60000/60000 [==============================] - 3s 56us/sample - loss: 159.8657 - acc: 0.1291
Epoch 9/30
60000/60000 [==============================] - 3s 57us/sample - loss: 159.8658 - acc: 0.1056
Epoch 10/30
60000/60000 [==============================] - 3s 56

## Stretch Goals:

- Use Hyperparameter Tuning to make the accuracy of your models as high as possible. (error as low as possible)
- Use Cross Validation techniques to get more consistent results with your model.
- Use GridSearchCV to try different combinations of hyperparameters. 
- Start looking into other types of Keras layers for CNNs and RNNs maybe try and build a CNN model for fashion-MNIST to see how the results compare.